# Scrape cases into MySQL DB

### Objective:
The goal for this task is to extract data (eg. text, HTML, links, attributes) of legal cases in webpage. After that, store it into a MySQL database.

### Task explaination
Visited https://www.cases.sheriahub.com/ to extract the following

- Ten cases listed.
- Extract case details.{eg. metadata, casetext}
- Store details into a database.


The database is a mysql database and the database table is has the following rows:
1. id – int - This is autoincrement so you will not insert anything.
2. case_title – varchar
3. metadata – json
4. case_judgement_html – MEDIUMTEXT
5. case_judgement_text– MEDIUMTEXT
6. pdf – LONGBLOB
7. date - datetime

In the case_title column, you will insert the case title (eg Muiruri Wanyoike v Utatuzi Center LLP & 20 others). In the metadata column, you will extract all the relevant metadata  (eg case number, parties, case class, judges, case court, etc) create a json and then insert the json. Eg
{
   "case_number":"Civil Appeal E036 of 2021",
   "parties":"Noor & another v Bank of Africa Limited & 2 others",
   "case_class":"civil",
   "judges":"A Mabeya"
}.
The case_judgement_html column will be the case judgement and will have the case judgement section as is including html. The case_judgement_text is the case judgement section without any html markup, you will strip any HTML tags from the text for this. You will download and upload the case pdf as a blob to the pdf section. Finally, date will be the datetime of insertion.

(Please note that in the future we will NOT be placing files into the database but this is for learning purposes only. Future files will be stored in cloud specifically AWS S3 buckets. Please research how to work with that in preparation for future work).

## Data Extraction

In [1]:
# load libraries
from urllib.request import urlopen as uReq;
from bs4 import BeautifulSoup as soup;
import json
import datetime

# page intrested to script
my_url = 'https://www.cases.sheriahub.com/'
# open connection and grab the page
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

# html parsing
page_soup = soup(page_html, "html.parser")

# grap all case
case_html = page_soup.findAll("div",{"class":"posttext pull-left"})
pdfs = page_soup.findAll("a",{"class":"btn btn-primary pull-right"})

#defined lists
case_titles= []
caseSub = [] #(eg. case/ede055b115)
pdf = []
dateList = []
case_judgement_html_List = [] 
case_judgement_text_List = [] 
metadataJsonList = []

# grap titles,pdfs,casesub
i = 0
while i<len(case_html):
    abc1 = case_html[i].a.text.strip()
    case_titles.append(abc1)
    abc2 = pdfs[i]["href"]
    pdf.append(abc2)
    abc3 = case_html[i].a["href"]
    caseSub.append(abc3)
    i+=1

# Grap Metadata,judgement HTML & text
for i in range(len(case_titles)):

    caseTableAtt = [] # list for columns of metadata table, clear the list
    caseTableValue = []# list for value of metadata table, clear the list
    metadatas = [] # list of metadata, clear the list
    metadata_json = '' # clear the variable
    
    
    # To get metadata inside the case
    url = str(my_url + caseSub[i])
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()

    
    # html parsing
    page_soup = soup(page_html, "html.parser")
    
    # metadata
    caseTables = page_soup.findAll("td")
    
    # case_text
    case_judgement_html = page_soup.find_all("span")[2:]
    case_judgement_html_List.append(case_judgement_html)
    case_judgement_text = soup(str(case_judgement_html), 'lxml').get_text().strip()
    case_judgement_text_List.append(case_judgement_text)
    # get date of scraping
    date = datetime.datetime.now()
    date = date.strftime("%y-%m-%d %H.%M.%S")
    dateList.append(date)
    
    
    j=0
    while j<=21:# 21 is number of tr of caseTables, fixed becuase some cases also have tr in the text
        
        caseTableAtt.append(caseTables[j].text)
        j+=1
        caseTableValue.append(caseTables[j].text.replace("\n",""))
        j+=1
        metadatas = dict(zip(caseTableAtt, caseTableValue))
    metadataJsonList.append(json.dumps(metadatas))


### Check stored data

In [2]:
case_titles

['Criminal Appeal 72 of 2018 - Republic v John Macharia Ndungu Mwari',
 'Criminal Appeal E026 of 2020 - Peter Gitonga Gituma v Republic',
 'Tribunal Case 5 of 2020 & 4 of 2021 ( Kisumu) (Consolidated) - Jack Ogola Ogolla v George Onyango Nyamor',
 'Environment and Land Court Suit 208 of 2014 - Daniel Nderitu Kagoko v Zuberi Mohammed Kombania, James Nyambane Masese, Brenda Bonarere Metobo, Registrar of Land, Kwale, Attorney General, Lawrence Kagoko Nderitu & Florence Nderitu',
 'Criminal Case (Murder) E002 of 2021 - Republic v John Waweru Ndungu & John Gitau Muriithi',
 'Criminal Appeal 19 of 2019 - Sammy Abiyo Jillo v Republic [2021] eKLR',
 'Environment and Land Case 226 of 2010 - Chesterton Properties Ltd v Samson Muturia & others',
 'Criminal Appeal 25 of 2019 - Abdi Aziz Maalim v Republic',
 'Environment and Land Suit 157 of 2017 - Mombasa Cement Limited v Salim Masoud Abdalla, Khalfan Abdalla, Rashid Mohamed Salim, Ahmed Mohamed Suleiman & Salim Al-Amin Suleiman',
 'Divorce Case  

In [3]:
caseSub

['case/d067ee5634/',
 'case/55f186cc62/',
 'case/a51580652b/',
 'case/964009c4bc/',
 'case/335054fc9a/',
 'case/adf78f8205/',
 'case/bdb53b49f6/',
 'case/7473bd06bf/',
 'case/867b40962b/',
 'case/fcf4524180/']

In [4]:
pdf

['https://www.cases.sheriahub.com/resources/pdf/d067ee5634/',
 'https://www.cases.sheriahub.com/resources/pdf/55f186cc62/',
 'https://www.cases.sheriahub.com/resources/pdf/a51580652b/',
 'https://www.cases.sheriahub.com/resources/pdf/964009c4bc/',
 'https://www.cases.sheriahub.com/resources/pdf/335054fc9a/',
 'https://www.cases.sheriahub.com/resources/pdf/adf78f8205/',
 'https://www.cases.sheriahub.com/resources/pdf/bdb53b49f6/',
 'https://www.cases.sheriahub.com/resources/pdf/7473bd06bf/',
 'https://www.cases.sheriahub.com/resources/pdf/867b40962b/',
 'https://www.cases.sheriahub.com/resources/pdf/fcf4524180/']

In [5]:
dateList

['21-09-23 01.20.21',
 '21-09-23 01.20.22',
 '21-09-23 01.20.23',
 '21-09-23 01.20.24',
 '21-09-23 01.20.25',
 '21-09-23 01.20.26',
 '21-09-23 01.20.27',
 '21-09-23 01.20.28',
 '21-09-23 01.20.29',
 '21-09-23 01.20.30']

In [6]:
case_judgement_text_List[4]

'[REPUBLIC OF KENYA, REPUBLIC OF KENYA, \xa0IN THE HIGH COURT OF KENYA , \xa0IN THE HIGH COURT OF KENYA , AT NAIVASHA, AT NAIVASHA, \xa0(CORAM: R. MWONGO, J), \xa0(CORAM: R. MWONGO, J), CRIMINAL CASE (MURDER) NO. E002 OF 2021, CRIMINAL CASE (MURDER) NO. E002 OF 2021, REPUBLIC..........................................................................................PROSECUTION, REPUBLIC..........................................................................................PROSECUTION, VERSUS, VERSUS, JOHN WAWERU NDUNGU................................................................1ST ACCUSED, JOHN WAWERU NDUNGU................................................................1ST ACCUSED, JOHN GITAU MURIITHI.....................................................................2nd ACCUSED, JOHN GITAU MURIITHI.....................................................................2nd ACCUSED, RULING , RULING , 1.\xa0 The 2nd Accused in this matter is jointly charged with the 1st Accused for th

In [7]:
metadataJsonList[4]

'{"Case Number": "Criminal Case (Murder) E002 of 2021", "Parties": "Republic v John Waweru Ndungu & John Gitau Muriithi", "Case Class": "Criminal", "Judges": "Richard Mururu Mwongo ", "Advocates": "Ms Maingi for the StateMr Wairegi for the 1st AccusedMr Owuor for the 2nd Accused John Waweru Ndungu - 1st Accused at Naivasha Medium PrisonJohn Gitau Muriithi - 2nd Accused at Naivasha Maximum Prison", "Case Action": "Ruling", "Case Outcome": "Matter fixed for hearing", "Date Delivered": "19 Jul 2021", "Court County": "Nakuru", "Case Court": "High Court at Naivasha", "Court Division": "Criminal"}'

## Connect to mySQL DB
Trying to connect to mysql DB and save data their

In [8]:
import mysql.connector

db = mysql.connector.connect(host= ,
                                         database=,
                                         user=,
                                         password=)


In [9]:
mycursor = db.cursor()
mycursor.execute("DESCRIBE ziyad_alshawi")
for x in mycursor:
    print(x)

('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment')
('case_title', 'varchar(225)', 'NO', '', None, '')
('metadata', 'json', 'NO', '', None, '')
('case_judgement_html', 'mediumtext', 'NO', '', None, '')
('case_judgement_text', 'mediumtext', 'NO', '', None, '')
('pdf', 'blob', 'NO', '', None, '')
('date', 'datetime', 'NO', '', None, '')


In [10]:
# insert into the database
for i in range(10):
    query = '''INSERT INTO `ziyad_alshawi` (`case_title`,`metadata`,`case_judgement_html`,`case_judgement_text`,`pdf`,`date`) VALUES ('{}','{}',"{}","{}","{}","{}")'''.format(case_titles[i],metadataJsonList[i] ,case_judgement_html_List[i],case_judgement_text_List[i],pdf[i],dateList[i])
    mycursor.execute(query)
    db.commit()

# Lesson Learned
- Data extraction is powerful tool to have as a data scientist.
- In DB records/rows need to enter in one query.
- In DB IDs usualy set on auto-increment to clear that need to use `truncate table table_name;` in MySQL Console.
- When extrate from text may encounter with `\n` inside the text which means in MySQL  ASCII 0x10 (newline) doesn't mean `\` + `n` expression. [source](https://stackoverflow.com/questions/47504402/mysql-invalid-json-text-invalid-escape-character-in-string)